In [1]:
import matplotlib.pyplot as plt      
from mpl_toolkits import mplot3d
import os
import sys
import glob
import time
import shutil
import gym
from gym import error, spaces, utils
import numpy as np
import math
from pathlib import Path

In [2]:
import gym_fish
from gym_fish.envs.lib import pyflare as fl

In [3]:
os.chdir(Path(os.getcwd()+"/../").resolve())
print(os.getcwd())

/root/pytorch_sac/test


In [4]:
cwd = os.getcwd()+"/"
data_folder =cwd+ 'py_data/'
json_folder =data_folder+'jsons/'
path_folder =json_folder+'paths/'
scripts_folder = cwd+'scripts/'
p_json = str(Path("./py_data/jsons/paths/line.json").resolve())
fluid_json = json_folder+'fluid_param_0.5.json'
rigid_json = json_folder+'rigids_4_30.json'
print(p_json,Path("./py_data/jsons/paths/line.json").exists())


/root/pytorch_sac/test/py_data/jsons/paths/line.json True


In [5]:
env=gym.make('fish-v0', 
             gpuId=3,couple_mode = fl.COUPLE_MODE.TWO_WAY,
                       path_json=p_json,
             frame_skip=200,
             rigid_json=rigid_json,
             fluid_json=fluid_json,
             wr=0.1
                   )

3
3
[0.97686249 2.00028078 2.99999994] [ 9.99980604e-01 -6.22825676e-03  2.04357705e-06] [2.97686249 2.00028078 2.99999994] [1. 0. 0.]


In [6]:

env1=gym.make('fish-v0', 
             gpuId=1,
                       path_json=p_json)
                   
env2=gym.make('fish-v0', 
             gpuId=2,
                       path_json=p_json
                   )
env0=gym.make('fish-v0', 
             gpuId=0,couple_mode = fl.COUPLE_MODE.TWO_WAY,
                       path_json=p_json,
             wr=0.1
                   )


1
1
[0.97686249 2.00028078 2.99999994] [ 9.99980604e-01 -6.22825676e-03  2.04357705e-06] [2.97686249 2.00028078 2.99999994] [1. 0. 0.]
2
2
[0.97686249 2.00028078 2.99999994] [ 9.99980604e-01 -6.22825676e-03  2.04357705e-06] [2.97686249 2.00028078 2.99999994] [1. 0. 0.]
0
0
[0.97686249 2.00028078 2.99999994] [ 9.99980604e-01 -6.22825676e-03  2.04357705e-06] [2.97686249 2.00028078 2.99999994] [1. 0. 0.]


In [7]:
env.stepOnce([0])
print(env.rigid_data.rigidWorld.getTimestep())
# env_test.save_data( save_fluid=True, save_objects=True,frame_num=0)
# check path, check object head and position ,data saving 
pd_controller = fl.PDControl1D(0.5,0.01,env.rigid_data.rigidWorld.time)

# env.plot3d()

0.0020000000949949026


In [8]:
def getPIDResponse():
    skeleton_dynamics = env.rigid_data.skeletons[0].dynamics
    pos = skeleton_dynamics.getBaseLink().getPosition()
    env.curPose_t = env.path_data.trajectory.getReferencePose(pos)
    target_pos = env.path_data.trajectory.getPose(env.curPose_t+0.05).getPosition()
    fwd = skeleton_dynamics.getBaseLinkFwd()
    fwd = fwd/np.linalg.norm(fwd)
    right = skeleton_dynamics.getBaseLinkRight()
    right = right/np.linalg.norm(right)
    expected_fwd = target_pos - pos
    expected_fwd = expected_fwd/np.linalg.norm( expected_fwd)
    
    angleShift = math.acos(fwd.dot(expected_fwd))
    if (expected_fwd - fwd).dot(-right)<=0:
        angleShift = -angleShift
#     offset = pd_controller.feedback(angleShift, env.rigid_data.rigidWorld.time)
    offset = 0
#     offset = 3.1415/6
    targetAngle = (3.1415/6)*math.sin(6.28*2*env.rigid_data.rigidWorld.time)+offset;
#     print((targetAngle-skeleton_dynamics.getJoint("spine02").getPositions()[0])/env.rigid_data.rigidWorld.getTimestep())
    return (targetAngle-skeleton_dynamics.getJoint("spine02").getPositions()[0])/env.rigid_data.rigidWorld.getTimestep()/5
#     return targetAngle/env.action_space_std

In [ ]:
def plot_infos(infos,title=None,reward_fig_name=None):
    dd = {k:[] for k in infos[0].keys()}
    for info in infos:
        for k in info.keys():
            dd[k].append(info[k])
    plt.figure()
    for k in dd.keys():
        if k=='action_penality':
            plt.plot(np.arange(0,len(dd[k])),[kkk  for kkk in dd[k]],label=k)
        else:
            plt.plot(np.arange(0,len(dd[k])),dd[k],label=k)
    plt.legend()
    if(title):
        plt.title(title)
    if(reward_fig_name):
        plt.savefig(reward_fig_name)
    plt.show()

In [ ]:
!./clean_visdata.sh

In [ ]:
import random
for i in range(1):
    env.reset()
    start_time = time.time()
    done = False
    infos = []
    skeleton_dynamics = env.rigid_data.skeletons[0].dynamics
    skeleton_dynamics.setHead(np.array([1,2,3]),np.array([0,0,-1]))
    while not done:
        act = getPIDResponse()
        obs,_,done,_=env.stepOnce(np.array([act,act,act,act]),save_fluid=True, save_objects=True)
#         obs,_,done,_=env.step(np.array([random.random()*2-1,random.random()*2-1,random.random()*2-1,random.random()*2-1]),save_fluid=False, save_objects=True)
#         infos.append(info)
#     plot_infos(infos)
    env.plot3d()
    end_time = time.time()
    print(end_time-start_time)

In [ ]:
env.rigid_data.rigidWorld.time